## Load course data

In [1]:
from learndata import LearnContents
ld = LearnContents()

## Fill the prerequisites list of each content that applies
It will not apply if the content has no siblings (subcontents of the same content)

In [2]:
#Fill prereq list of each content

#for lc in ld:
    #if not lc.parent == None:
        #continue
    
    #done_subcontents = list()
    
    #for subcont in lc:
        #subcont.prereqs = list(done_subcontents)
        #done_subcontents.append(subcont)

In [3]:
#Topics pre requisites will be every topic that is cited before it 

#We may take diferent approachs for diferent subsections levels, 
#like place as prereq every subsection level above the current target 
#or only the the subsections of the same level

#for course in udemy_data: #Iterate thru every course
for lc in ld:
    #Work only with top contents (with no parents)
    if not lc.parent == None:
        continue
    
    #Lists to register the current course contents that already happend in subsecs degrees 1 and 2
    course_subsec1_contents = list() 
    course_subsec2_contents = list()    
    
    for subsec1_content in lc: #Iterate thru content
        #We may option to not include excluded topics later (but best way is to apply probabilistic models to it too)
        subsec1_content.prereqs = list(course_subsec1_contents) #For the current, append everything before    
        course_subsec1_contents.append(subsec1_content)

        #Analog to sub sec 2
        for subsec2_content in subsec1_content:
            subsec2_content.prereqs = list(course_subsec2_contents)
            course_subsec2_contents.append(subsec2_content)

In [5]:
#ld[995].prereqs

## Create similarity functions

In [6]:
from nltk.stem import LancasterStemmer,PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from string import punctuation

stemmer = PorterStemmer()
#stemmer = LancasterStemmer()

stop_words = list(punctuation)# + stopwords.words("english")

def get_tokens(sentence):
    return set([stemmer.stem(token) for token in word_tokenize(sentence) if token not in stop_words]) 

def stem_tokens_similarity(lc1, lc2):
    """
    Function to get similiarity of two contents getting
    the jaccard similarity of the stemmed tokens of the words.
    """
    
    #MUST PRE CREATE A DICT WITH STEMMED TERMS TO SEARCH
    
    #Function to compute similarity between two learn contents.
    #This may compare their childs, titles, anything.
    #For now, get the jaccard similarity of the terms in the title.
    
    #Returns: float value between 0 and 1
    
    lc1_tokens = get_tokens(str(lc1).lower())
    lc2_tokens = get_tokens(str(lc2).lower())
    
    #Jaccard similarity: |intersection| over |union|
    
    return len(lc1_tokens.intersection(lc2_tokens)) / len(lc1_tokens.union(lc2_tokens))

def case_insensitive_word_match_similarity(lc1, lc2):
    word1 = str(lc1).lower()
    word2 = str(lc2).lower()
    if word1 == word2:
        return 1.0
    else:
        return 0.0
    
assert case_insensitive_word_match_similarity("lucas", "Lucas") == 1.0
assert case_insensitive_word_match_similarity("lucass", "Lucas") == 0.0

In [35]:
def query(term, similarity_func=case_insensitive_word_match_similarity, cut_score=1.0):
    """Function to query content based on a term, similarity function and cut_score."""
    
    results = list()
    
    for lc in ld:
        similarity_score = similarity_func(term, lc)
        if similarity_score >= cut_score:
            results.append((lc, similarity_score))
    
    return results
    
    
    term_tokens = term.lower().split()
    idx_set = set()
    for tok in term_tokens:
        for _,_,idx in ld.search(tok):
            idx_set.add(idx)
        
    results = list()
    for idx in idx_set:
        results.append((ld[idx], similarity_func(term, ld[idx])))
        
    results = [(result, score) for result, score in results if score >= cut_score]
        
    return sorted(results, key=lambda a:a[1], reverse=True)    

In [36]:
query_results = query("recurrent neural networks")

In [42]:
query_results

[(Recurrent Neural Networks, 1.0),
 (Recurrent neural networks, 1.0),
 (Recurrent Neural Networks, 1.0),
 (Recurrent neural networks, 1.0),
 (Recurrent Neural Networks, 1.0),
 (Recurrent Neural Networks, 1.0),
 (Recurrent Neural Networks, 1.0),
 (Recurrent Neural Networks, 1.0),
 (Recurrent Neural Networks, 1.0)]

In [76]:
from collections import Counter

def get_term_prereqs(term, similarity_func, cut_score):
    #1. Query contents that match the specified term with the similarity function.
    query_results = query(term, similarity_func, cut_score)
    
    #2. Group all prereqs with their scores
    raw_prereqs = list()
    for cont, score in query_results:
        if not hasattr(cont, 'prereqs'):
            continue
        for pr in cont.prereqs:
            raw_prereqs.append((pr, score))
            
    #3. Get n similar content to the prereqs based in a cut value (TO BE DONE) 
            
    #4. Apply loopback to all prereqs
    non_loopback_prereqs = list()
    for pr, pr_score in raw_prereqs:
        if not hasattr(pr, 'prereqs'):
            non_loopback_prereqs.append((pr, pr_score))
            continue
            
        max_term_pr_similarity = 0
            
        #Iterate thru the prereqs of the prereq
        for pr_pr in pr.prereqs:
            term_pr_similarity = similarity_func(term, pr_pr)
            max_term_pr_similarity = max(max_term_pr_similarity, term_pr_similarity)
            
        #Subtract the max term_prereq similarity from the pr_score to compute the score after loopback
        pr_score -= max_term_pr_similarity
        
        #If the pr_score is bigger than 0, mean the pr is more prereq of the query than the inverse
        if pr_score > 0:
            non_loopback_prereqs.append((pr, pr_score))
            
            
    #5. Merge results for presentation
    #We can do merge by:
    #  -Matching terms
    #  -Checking similarities between then
    #  -Creating clusters based on features such title terms
    #  etc
    
    #Merge simpling matching terms
    merged_prereqs = Counter([str(pr).lower() for pr,_ in non_loopback_prereqs])
        
    return merged_prereqs.most_common()

In [88]:
pr_list = get_term_prereqs("generics", case_insensitive_word_match_similarity, 1)
print(len(pr_list))
print("\n".join(map(str,pr_list)))

proper implement stem token similarity
implement step 3 to get similar prereqs
think a better approach for the loopback max term_pr_similarity

1516
('section introduction', 55)
('section summary', 52)
('summary', 36)
('introduction', 20)
('interfaces', 13)
('functions', 12)
('classes', 11)
('objectives', 11)
('module introduction', 10)
('inheritance', 10)
('labs', 9)
('extensions', 9)
('arrays', 8)
('chapter introduction', 7)
('section details', 6)
('variables', 6)
('wrap up', 5)
('methods', 5)
('enums', 5)
('abstract classes', 5)
('protocols', 5)
('the course overview', 5)
('scope', 4)
('for loops', 4)
('strings', 4)
('closures', 4)
('packages', 4)
('collections', 4)
('operators', 4)
('string interpolation', 4)
('optional chaining', 4)
('classes and objects', 3)
('basic types', 3)
('course introduction', 3)
('lists', 3)
('arithmetic operators', 3)
('functional programming', 3)
('enumerations', 3)
('maps', 3)
('booleans', 3)
('control flow', 3)
('wrapper classes', 3)
('section conclusion', 3)
('lab 2 review', 3)
('access control', 3)
('lab 1 review', 3)
('course overview', 3)
('enum', 3)
('lab 3 review', 3)
('polymorphism', 3